In [14]:
import sqlite3
conn = sqlite3.connect('database.db')
c = conn.cursor()
sql = """
DROP TABLE IF EXISTS users;
CREATE TABLE users (
           id integer unique primary key autoincrement,
           name text,
           work text
);
"""
c.executescript(sql)
conn.commit()
conn.close()



In [17]:
# ask for person name and profession
#collect 20 picture of faces then convert it to grayscale
import cv2
import numpy as np 
import sqlite3
import os
conn = sqlite3.connect('database.db')
if not os.path.exists('./dataset'):
    os.makedirs('./dataset')
c = conn.cursor()
face_cascade = cv2.CascadeClassifier(r'C:\Users\User\opencv\data\haarcascades/haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)
uname = input("Enter your name: ")
uwork = input("Enter your profession: ")
c.execute('INSERT INTO users (name, work) VALUES (?,?)', (uname, uwork))
uid = c.lastrowid
sampleNum = 0
while True:
  ret, img = cap.read()
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  faces = face_cascade.detectMultiScale(gray, 1.3, 5)
  for (x,y,w,h) in faces:
    sampleNum = sampleNum+1
    cv2.imwrite("dataset/User."+str(uid)+"."+str(sampleNum)+".jpg",gray[y:y+h,x:x+w])
    cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0), 2)
    cv2.waitKey(100)
  cv2.imshow('img',img)
  cv2.waitKey(1);
    #
  if sampleNum > 20:
    break
cap.release()
conn.commit()
conn.close()
cv2.destroyAllWindows()

Enter your name: najihah
Enter your profession: accountant


In [18]:
#recognizer
#convert dataset to yml file 
import os
import numpy as np
import cv2
from PIL import Image # For face recognition we will the the LBPH Face Recognizer 
recognizer = cv2.face.LBPHFaceRecognizer_create() 
path = 'dataset'
if not os.path.exists('./recognizer'):
    os.makedirs('./recognizer')
def getImagesWithID(path):
    imagePaths = [os.path.join(path, f) for f in os.listdir(path)]   
 # print image_path   
 # getImagesWithID(path)
    faces = []
    IDs = []
    for imagePath in imagePaths:      
  # Read the image and convert to grayscale
        facesImg = Image.open(imagePath).convert('L')
        faceNP = np.array(facesImg, 'uint8')
        # Get the label of the image
        ID= int(os.path.split(imagePath)[-1].split(".")[1])
         # Detect the face in the image
        faces.append(faceNP)
        IDs.append(ID)
        cv2.imshow("Adding faces for traning",faceNP)
        cv2.waitKey(10)
    return np.array(IDs), faces
Ids,faces  = getImagesWithID(path)
recognizer.train(faces,Ids)
recognizer.save('recognizer/trainingData.yml')
cv2.destroyAllWindows()

In [19]:
#code for detector
#compare the picture with image in database
#display name and profession
import cv2
import numpy as np 
import sqlite3
import os
conn = sqlite3.connect('database.db')
c = conn.cursor()
d = conn.cursor()
fname = "recognizer/trainingData.yml"
if not os.path.isfile(fname):
  print("Please train the data first")
  exit(0)
face_cascade = cv2.CascadeClassifier(r'C:\Users\User\opencv\data\haarcascades/haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read(fname)
while True:
  ret, img = cap.read()
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  faces = face_cascade.detectMultiScale(gray, 1.3, 5)
  for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
    ids,conf = recognizer.predict(gray[y:y+h,x:x+w])
    c.execute("select name from users where id = (?);", (ids,))
    d.execute("select work from users where id = (?);", (ids,))
    result = c.fetchall()
    result2 =d.fetchall()
    name = result[0][0]
    work = result2[0][0]
    if conf < 50:
      cv2.putText(img, name, (x+2,y+h-25), cv2.FONT_HERSHEY_SIMPLEX, 1, (150,255,0),2)
      cv2.putText(img, work, (x+2,y+h-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (150,255,0),2)
    else:
      cv2.putText(img, 'Unidentified', (x+2,y+h-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),2)
  cv2.imshow('Face Recognizer',img)
  k = cv2.waitKey(30) & 0xff
  if k == 27:
    break
cap.release()
cv2.destroyAllWindows()